# Import Library

In [1]:
from tqdm import tqdm
import time
import joblib
import warnings
import os

import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostRegressor

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

%reload_ext autotime

time: 0 ns (started: 2024-08-03 02:13:59 +09:00)


# Load Data

In [2]:
# 기본 경로 설정
base_path = './bami/data/total/'

# 파일명 리스트
file_names = ['방문지정보.csv', '여행.csv', '여행객 Master.csv']

# 데이터프레임 딕셔너리에 로드
dataframes = {file_name: pd.read_csv(os.path.join(base_path, file_name)) for file_name in file_names}

# 각각의 데이터프레임 할당
visit_area_info = dataframes['방문지정보.csv']
travel = dataframes['여행.csv']
traveler_master = dataframes['여행객 Master.csv']

time: 344 ms (started: 2024-08-03 02:13:59 +09:00)


# Preprocessing

## visit_area_info 방문지 정보

In [3]:
# 관광지 선택
# 자연관광지, 역사/유적/종교 시설(문화재, 박물관, 촬영지, 절 등), 문화 시설(공연장, 영화관, 전시관 등), 상업지구(거리, 시장, 쇼핑시설)
# 레저/스포츠 관련 시설(스키, 카트, 수상레저), 테마시설(놀이공원, 워터파크), 산책로, 둘레길 등, 지역 축제/행사
visit_area_info = visit_area_info.loc[visit_area_info['VISIT_AREA_TYPE_CD'].between(1, 8)]

# LOTNO_ADDR (지번주소) 열에 결측값이 있는 행 제거 후 인덱스 재설정
# ex) 경기 수원시 팔달구 매향동 3-32
visit_area_info = visit_area_info.dropna(subset=['LOTNO_ADDR']).reset_index(drop=True)

# 데이터 확인
visit_area_info.head()

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,ROAD_NM_CD,LOTNO_CD,POI_ID,POI_NM,RESIDENCE_TIME_MIN,VISIT_AREA_TYPE_CD,REVISIT_YN,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD,region
0,2304300002,e_e000004,2,화성 관광열차 안내소 연무대 매표소,2023-04-30,2023-04-30,경기 수원시 팔달구 창룡대로103번길 20,경기 수원시 팔달구 매향동 3-32,127.023339,37.287878,NaN,NaN,POI01000000ALZU7R,동대문종합시장 악세서리부자재시장,60.0,2,N,10.0,NaN,4.0,3.0,4.0,NaN,capital
1,2304300003,e_e000004,3,창룡문,2023-04-30,2023-04-30,NaN,경기 수원시 팔달구 남수동,127.025143,37.287791,NaN,NaN,POI010000006N1USC,창룡문,30.0,2,N,1.0,NaN,4.0,4.0,4.0,NaN,capital
2,2304300004,e_e000004,4,수원 화성 화홍문,2023-04-30,2023-04-30,NaN,경기 수원시 팔달구 북수동 9000-1,127.017626,37.287546,NaN,NaN,POI01000TR021821V,수원화성 화홍문,60.0,2,N,10.0,NaN,4.0,3.0,3.0,NaN,capital
3,2304300004,e_e000006,4,경춘선 자전거길,2023-04-30,2023-04-30,NaN,경기 가평군 청평면 하천리 158-2,127.436200,37.745958,NaN,NaN,POI01000TR008470V,경춘선자전거길,150.0,1,Y,4.0,NaN,5.0,5.0,5.0,NaN,capital
4,2304290002,e_e000009,2,농협안성팜랜드,2023-04-29,2023-04-29,경기 안성시 공도읍 대신두길 28,경기 안성시 공도읍 신두리 451,127.193517,36.991317,NaN,NaN,POI01000000E66UWC,안성팜랜드,30.0,6,N,1.0,NaN,4.0,4.0,4.0,NaN,capital


time: 47 ms (started: 2024-08-03 02:13:59 +09:00)


In [4]:
# 시도/군구 변수 생성
visit_area_info['SIDO'] = visit_area_info['LOTNO_ADDR'].apply(lambda x: x.split(' ')[0])
visit_area_info['GUNGU'] = visit_area_info['LOTNO_ADDR'].apply(lambda x: x.split(' ')[1])

time: 16 ms (started: 2024-08-03 02:13:59 +09:00)


In [5]:
visit_area_info['SIDO'].value_counts()

SIDO
경기         3232
서울         2328
경북         2223
전북         1719
충남         1688
강원특별자치도    1662
전남         1660
부산         1552
강원         1178
대전         1143
충북         1124
경남         1028
인천          571
대구          441
광주          295
울산          273
세종특별자치시     218
강원도           5
경상북도          4
전라북도          3
경기도           3
부산광역시         3
충청남도          3
전라남도          2
충청북도          2
서울특별시         1
경상남도          1
Name: count, dtype: int64

time: 0 ns (started: 2024-08-03 02:13:59 +09:00)


### 변수 선택
- TRAVEL_ID: 여행 ID
- VISIT_AREA_NM: 방문 장소 이름
- SIDO: 시/도
- GUNGU: 군/구
- VISIT_AREA_TYPE_CD: 관광 장소 유형
- DGSTFN: 만족도
- REVISIT_INTENTION: 재방문의향
- RCMDTN_INTENTION: 추천의향
- RESIDENCE_TIME_MIN: 체류시간분
- REVISIT_YN: 재방문여부

In [6]:
visit_area_info = visit_area_info[['TRAVEL_ID', 'VISIT_AREA_NM', 'SIDO', 'GUNGU', 'VISIT_AREA_TYPE_CD', 'DGSTFN',
                                   'REVISIT_INTENTION', 'RCMDTN_INTENTION', 'RESIDENCE_TIME_MIN', 'REVISIT_YN']]

visit_area_info.head()

,TRAVEL_ID,VISIT_AREA_NM,SIDO,GUNGU,VISIT_AREA_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,RESIDENCE_TIME_MIN,REVISIT_YN
0,e_e000004,화성 관광열차 안내소 연무대 매표소,경기,수원시,2,4.0,3.0,4.0,60.0,N
1,e_e000004,창룡문,경기,수원시,2,4.0,4.0,4.0,30.0,N
2,e_e000004,수원 화성 화홍문,경기,수원시,2,4.0,3.0,3.0,60.0,N
3,e_e000006,경춘선 자전거길,경기,가평군,1,5.0,5.0,5.0,150.0,Y
4,e_e000009,농협안성팜랜드,경기,안성시,6,4.0,4.0,4.0,30.0,N


time: 0 ns (started: 2024-08-03 02:13:59 +09:00)


## travel 여행 정보

In [7]:
# TRAVEL_MISSION_CHECK (미션 우선도)의 첫번째 항목 가져오기
travel['TRAVEL_MISSION_PRIORITY'] = travel['TRAVEL_MISSION_CHECK'].str.split(';').apply(lambda x: x[0])

time: 0 ns (started: 2024-08-03 02:13:59 +09:00)


### 변수 선택
- TRAVEL_ID: 여행 ID
- TRAVELER_ID: 여행자 ID
- TRAVEL_MISSION_PRIORITY: 개별 미션 우선도 중 첫 번째

In [8]:
travel = travel[['TRAVEL_ID', 'TRAVELER_ID', 'TRAVEL_MISSION_PRIORITY']]

time: 0 ns (started: 2024-08-03 02:13:59 +09:00)


## traveler_master 여행자 정보

### 변수 선택
- TRAVELER_ID: 여행자 ID
- GENDER: 성별
- AGE_GRP: 연령대
- INCOME: 소득
- TRAVEL_STYL_{N}: 여행 스타일
- TRAVEL_MOTIVE_1: 여행 동기 (2, 3은 결측치가 있어서 제외)
- TRAVEL_NUM: 여행빈도
- TRAVEL_COMPANIONS_NUM: 동반자 수

In [9]:
columns_needed = [
    'TRAVELER_ID', 'GENDER', 'AGE_GRP', 'INCOME', 'TRAVEL_STYL_1',
    'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5',
    'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'TRAVEL_MOTIVE_1', 'TRAVEL_NUM', 'TRAVEL_COMPANIONS_NUM'
]

# 필요한 열만 선택하여 데이터프레임 생성
traveler_master = traveler_master[columns_needed]

time: 0 ns (started: 2024-08-03 02:13:59 +09:00)


## 데이터프레임 합치기 

In [10]:
# 먼저 travel과 traveler_master 데이터를 병합
merged_travel_data = pd.merge(travel, traveler_master, on='TRAVELER_ID', how='inner')

# visit_area_info 데이터와 병합
df = pd.merge(visit_area_info, merged_travel_data, on='TRAVEL_ID', how='right')

time: 31 ms (started: 2024-08-03 02:13:59 +09:00)


In [11]:
df

,TRAVEL_ID,VISIT_AREA_NM,SIDO,GUNGU,VISIT_AREA_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,RESIDENCE_TIME_MIN,REVISIT_YN,TRAVELER_ID,TRAVEL_MISSION_PRIORITY,GENDER,AGE_GRP,INCOME,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_NUM,TRAVEL_COMPANIONS_NUM
0,e_e000004,화성 관광열차 안내소 연무대 매표소,경기,수원시,2.0,4.0,3.0,4.0,60.0,N,e000004,3,남,40,7,5,3,5,4,5,4,2,5,1,1,2
1,e_e000004,창룡문,경기,수원시,2.0,4.0,4.0,4.0,30.0,N,e000004,3,남,40,7,5,3,5,4,5,4,2,5,1,1,2
2,e_e000004,수원 화성 화홍문,경기,수원시,2.0,4.0,3.0,3.0,60.0,N,e000004,3,남,40,7,5,3,5,4,5,4,2,5,1,1,2
3,e_e000006,경춘선 자전거길,경기,가평군,1.0,5.0,5.0,5.0,150.0,Y,e000006,21,남,30,5,3,3,2,4,5,4,3,2,1,4,1
4,e_e000009,농협안성팜랜드,경기,안성시,6.0,4.0,4.0,4.0,30.0,N,e000009,22,여,30,4,3,2,3,3,2,6,4,7,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22784,h_h001459,단양구경시장,충북,단양군,4.0,5.0,5.0,5.0,30.0,N,h001459,22,남,30,4,4,2,2,3,2,5,1,7,7,1,1
22785,h_h001459,도담삼봉,충북,단양군,1.0,5.0,5.0,5.0,30.0,N,h001459,22,남,30,4,4,2,2,3,2,5,1,7,7,1,1
22786,h_h003280,법주사,충북,보은군,2.0,5.0,5.0,5.0,90.0,N,h003280,3,여,40,5,1,4,4,4,4,4,4,4,4,1,0
22787,h_h003280,정이품송,충북,보은군,1.0,5.0,5.0,5.0,30.0,N,h003280,3,여,40,5,1,4,4,4,4,4,4,4,4,1,0


time: 16 ms (started: 2024-08-03 02:13:59 +09:00)


In [12]:
df['TRAVEL_ID'].nunique()

7680

time: 15 ms (started: 2024-08-03 02:13:59 +09:00)


## 만족도(y) 결측치 삭제

In [13]:
df = df.dropna(subset=['DGSTFN']).reset_index(drop=True)

time: 16 ms (started: 2024-08-03 02:13:59 +09:00)


In [14]:
df['TRAVEL_ID'].nunique()

7253

time: 0 ns (started: 2024-08-03 02:13:59 +09:00)


## 체류시간 결측치 대체

체류시간 0을 median 60으로 바꾸기

In [15]:
df['RESIDENCE_TIME_MIN'] = df['RESIDENCE_TIME_MIN'].replace(0, 60)

time: 0 ns (started: 2024-08-03 02:13:59 +09:00)


## 재방문여부 원핫인코딩

In [16]:
df['REVISIT_YN'] = np.where(df['REVISIT_YN'] == 'N', 0, 1)

time: 16 ms (started: 2024-08-03 02:14:00 +09:00)


### 여행스타일 결측치 삭제

In [17]:
df = df.dropna(subset=['TRAVEL_STYL_1']).reset_index(drop=True)

time: 15 ms (started: 2024-08-03 02:14:00 +09:00)


### 여행 목적 결측치 삭제

In [18]:
# 결측치 저장 및 초기화
df = df.reset_index(drop=True)
missing = df[df['TRAVEL_MOTIVE_1'].isna()].copy()

# 최빈값으로 채우기 (1: 쇼핑)
missing['TRAVEL_MOTIVE_1'] = 1.0

# 결측치 채운 데이터프레임을 원래 데이터프레임에 반영
df.update(missing)

# 결측값이 채워졌는지 확인하기 위해 TRAVEL_MOTIVE_1 열의 결측값 제거 후 인덱스 재설정
df = df.dropna(subset=['TRAVEL_MOTIVE_1']).reset_index(drop=True)

time: 16 ms (started: 2024-08-03 02:14:00 +09:00)


In [19]:
df['TRAVEL_MOTIVE_1'].value_counts()

TRAVEL_MOTIVE_1
1     5851
3     5688
2     4525
7     2202
8     1332
5      711
6      643
4      620
9      543
10     247
Name: count, dtype: int64

time: 0 ns (started: 2024-08-03 02:14:00 +09:00)


In [20]:
df.shape

(22362, 26)

time: 15 ms (started: 2024-08-03 02:14:00 +09:00)


In [21]:
df.isna().sum().sum()

0

time: 0 ns (started: 2024-08-03 02:14:00 +09:00)


In [22]:
df['TRAVEL_ID'].nunique()

7253

time: 0 ns (started: 2024-08-03 02:14:00 +09:00)


# Train-test split

**고려사항**

1. 유니크한 관광지 정보가 모두 train set에 있어야 한다.
  - i.e. 모델을 학습할 때 모든 관광지에 대한 정보를 학습해야 한다. 따라서, 훈련 세트에 포함되지 않은 관광지는 테스트 세트에도 포함되지 않아야 한다.
  - $\because$ 방문지 변수를 사용하기 때문에 모델이 학습하지 않은 방문지에 대한 정보를 예측할 수 없음
  - e.g. 경복궁에 대한 정보가 훈련 세트에 없으면 테스트 세트에도 경복궁이 포함되면 안 된다.
2. 새로운 유저에 대한 추측이기 때문에 유저 데이터는 무조건 train/test 중 한 곳에만 있어야 한다.
  - i.e. 한 유저의 데이터는 훈련 세트 또는 테스트 세트 중 하나에만 포함되어야 한다. 한 유저의 관광지 정보가 훈련 세트와 테스트 세트에 동시에 있으면 안 된다.
  - $\because$ 새로운 유저에 대한 예측을 진행하는 경우, 한 유저의 일부 데이터가 훈련 세트와 테스트 세트에 동시에 존재하면 모델 평가의 공정성이 떨어질 수 있음
  - e.g. 유저 A가 경복궁을 방문한 기록이 훈련 세트에 있으면, 유저 A의 다른 방문 기록(예: 북촌 한옥마을)은 테스트 세트에 포함되지 않아야 한다.
3. 이를 반영해서 train/test split을 진행하면 된다.
  - i.e. 1과 2의 조건을 반영하여 데이터셋을 훈련 세트와 테스트 세트로 나누어야 한다.
  - 방법
    1. 모든 유니크한 관광지 정보를 훈련 세트에 포함시킨다.
    2. 각 유저의 데이터를 훈련 세트와 테스트 세트 중 하나에만 포함되도록 분할한다.
4. train set에서만 특정 변수들의 평균을 산출하고 이를 test set에 대입한다.
   - i.e. 훈련 세트에서 특정 변수들(e.g. 체류시간, 추천 의향, 재방문 여부, 동반자 수, 재방문 의향)의 평균을 계산하고, 이 값을 테스트 세트에 대입한다.
   - $\because$ 훈련 세트에서 얻은 평균 값을 테스트 세트에 적용함으로써 모델이 훈련 세트에서 학습한 내용을 테스트 세트에 반영할 수 있다.
   - 방법
     1. 훈련 세트에서 각 관광지마다 체류시간 평균, 추천 의향 평균, 재방문 여부 평균, 동반자 수 평균, 재방문 의향 평균을 계산한다.
     2. 이 값을 테스트 세트의 해당 관광지에 대입한다. 

In [23]:
df['VISIT_AREA_NM'].unique()

array(['화성 관광열차 안내소 연무대 매표소', '창룡문', '수원 화성 화홍문', ..., '농협 하나로마트 목포점',
       '유달산 주차장', '문의 청남대 휴게소 청주 방향'], dtype=object)

time: 0 ns (started: 2024-08-03 02:14:00 +09:00)


이 프로세스를 반복하면 전체 데이터프레임(df1)에 유저정보는 계속 삭제될 것이고, 남은 df1가 test set, Train 데이터프레임이 train set이 되는 것

In [24]:
# df 복사본 생성
df1 = df.copy()
Train = pd.DataFrame(columns=df.columns)

# 유니크한 관광지 목록
unique_areas = df['VISIT_AREA_NM'].unique()

np.random.seed(42)

for area in tqdm(unique_areas):
    # 특정 관광지에 간 모든 사람을 뽑아서
    df2 = df1[df1['VISIT_AREA_NM'] == area]
    if not df2.empty:
        # 랜덤으로 한 명의 TRAVEL_ID를 뽑음
        random_id = df2.sample(1, random_state=42)['TRAVEL_ID'].values[0]
        # 그 사람이 간 모든 관광지를 구해서
        df3 = df1[df1['TRAVEL_ID'] == random_id]
        # TRAIN 데이터프레임에 추가
        Train = pd.concat([Train, df3])
        # df1에서 제거
        df1 = df1[df1['TRAVEL_ID'] != random_id]
        
# TRAIN 데이터셋이 전체 데이터셋의 80%가 될 때까지 샘플링 반복
while len(df1) / len(df) > 0.2:
    random_id = df1.sample(1, random_state=42)['TRAVEL_ID'].values[0]
    df3 = df1[df1['TRAVEL_ID'] == random_id]
    Train = pd.concat([Train, df3])
    df1 = df1[df1['TRAVEL_ID'] != random_id]

100%|██████████| 8805/8805 [00:36<00:00, 242.70it/s]

time: 36.3 s (started: 2024-08-03 02:14:00 +09:00)


In [25]:
# 데이터셋 크기 및 비율 출력
train_size = len(Train)
test_size = len(df1)
total_size = len(df)
test_ratio = test_size / total_size

print(f"Train set의 크기: {train_size}, Test set의 크기: {test_size}")
print(f"전체 data의 크기: {total_size}")
print(f"Test set의 비율: {test_ratio:.2f}")

Train set의 크기: 17958, Test set의 크기: 4404
전체 data의 크기: 22362
Test set의 비율: 0.20
time: 0 ns (started: 2024-08-03 02:14:36 +09:00)


## Train set에서 방문지에 대한 변수 생성

방문지마다 체류시간 평균, 추천의향의 평균, 재방문여부의 평균, 동반자 수의 평균, 재방문의향의 평균 산출

In [26]:
# 필요한 평균값을 계산하고 병합
means = Train.groupby('VISIT_AREA_NM').agg({
    'RESIDENCE_TIME_MIN': 'mean',
    'RCMDTN_INTENTION': 'mean',
    'REVISIT_YN': 'mean',
    'TRAVEL_COMPANIONS_NUM': 'mean',
    'REVISIT_INTENTION': 'mean'
}).rename(columns={
    'RESIDENCE_TIME_MIN': 'RESIDENCE_TIME_MIN_mean',
    'RCMDTN_INTENTION': 'RCMDTN_INTENTION_mean',
    'REVISIT_YN': 'REVISIT_YN_mean',
    'TRAVEL_COMPANIONS_NUM': 'TRAVEL_COMPANIONS_NUM_mean',
    'REVISIT_INTENTION': 'REVISIT_INTENTION_mean'
})

# 새로운 데이터프레임 생성
new_train = Train.merge(means, on='VISIT_AREA_NM', how='left')

time: 953 ms (started: 2024-08-03 02:14:36 +09:00)


In [27]:
new_train = new_train.sort_values(by=['TRAVEL_ID'], axis=0)

time: 16 ms (started: 2024-08-03 02:14:37 +09:00)


## Data Set 저장

In [28]:
path = './'

# train set 저장
new_train.to_csv(os.path.join(path, '관광지 추천시스템 Trainset.csv'), index=False)

# test set 저장
df1.to_csv(os.path.join(path, '관광지 추천시스템 Testset.csv'), index=False)

time: 187 ms (started: 2024-08-03 02:14:37 +09:00)


In [29]:
# 파일 불러오기
train = pd.read_csv(os.path.join(path, "관광지 추천시스템 Trainset.csv"))
test = pd.read_csv(os.path.join(path, "관광지 추천시스템 Testset.csv"))

time: 78 ms (started: 2024-08-03 02:14:37 +09:00)


In [30]:
train = train.dropna().reset_index(drop=True)
test = test.dropna().reset_index(drop=True)

time: 0 ns (started: 2024-08-03 02:14:37 +09:00)


In [31]:
print(train.shape)
print(test.shape)

(17958, 31)
(4404, 26)
time: 0 ns (started: 2024-08-03 02:14:37 +09:00)


# Train set 여행 방문지 필터링

## n번 이상 반복한 곳에 대해서만 학습/테스트

In [32]:
count = pd.DataFrame(train['VISIT_AREA_NM'].value_counts())

count

,count
VISIT_AREA_NM,
광안리해수욕장,106
해운대해수욕장,84
전주한옥마을,82
대릉원,71
황련단길,70
...,...
커피 플레이스,1
원주 로데오거리,1
캠핑 바비큐 한마당,1


time: 16 ms (started: 2024-08-03 02:14:37 +09:00)


In [33]:
count['count'].values

array([106,  84,  82, ...,   1,   1,   1], dtype=int64)

time: 16 ms (started: 2024-08-03 02:14:37 +09:00)


In [34]:
train = train.reset_index(drop=True)

# 4번 이상 방문한 곳으로 필터링
count = train['VISIT_AREA_NM'].value_counts()
four_places = count[count >= 2].index

# 필터링
train = train[train['VISIT_AREA_NM'].isin(four_places)].reset_index(drop=True)

time: 15 ms (started: 2024-08-03 02:14:37 +09:00)


In [35]:
train['VISIT_AREA_NM'].value_counts().value_counts().index

Index([  2,   3,   4,   5,   6,   7,   8,   9,  10,  12,  11,  15,  14,  13,
        16,  17,  21,  23,  19,  22,  18,  25,  20,  30,  24,  36,  31,  27,
        26,  28,  34,  37,  51,  82,  71,  70,  66,  60,  58,  56,  54,  45,
        50,  84,  43,  42,  41,  38,  33,  32, 106],
      dtype='int64', name='count')

time: 0 ns (started: 2024-08-03 02:14:37 +09:00)


In [36]:
print(train.shape)
print(test.shape)

(11673, 31)
(4404, 26)
time: 0 ns (started: 2024-08-03 02:14:37 +09:00)


In [37]:
(train['TRAVEL_ID'].nunique() + test['TRAVEL_ID'].nunique())

6398

time: 0 ns (started: 2024-08-03 02:14:37 +09:00)


# CatBoost 모델 기반 추천시스템 학습

In [38]:
drop_columns = ['TRAVELER_ID', 'REVISIT_INTENTION', 'RCMDTN_INTENTION', 'RESIDENCE_TIME_MIN', 'REVISIT_YN']

train = train.drop(columns=drop_columns)
test = test.drop(columns=drop_columns)

time: 0 ns (started: 2024-08-03 02:14:37 +09:00)


In [39]:
# 데이터 타입 변경
train['VISIT_AREA_TYPE_CD'] = train['VISIT_AREA_TYPE_CD'].astype(str)
test['VISIT_AREA_TYPE_CD'] = test['VISIT_AREA_TYPE_CD'].astype(str)

time: 16 ms (started: 2024-08-03 02:14:37 +09:00)


In [40]:
X_train = Train.drop(columns=['DGSTFN'], axis=1)
y_train = Train['DGSTFN']

time: 15 ms (started: 2024-08-03 02:14:37 +09:00)


In [41]:
X_train

,TRAVEL_ID,VISIT_AREA_NM,SIDO,GUNGU,VISIT_AREA_TYPE_CD,REVISIT_INTENTION,RCMDTN_INTENTION,RESIDENCE_TIME_MIN,REVISIT_YN,TRAVELER_ID,TRAVEL_MISSION_PRIORITY,GENDER,AGE_GRP,INCOME,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_NUM,TRAVEL_COMPANIONS_NUM
311,e_e000541,화성 관광열차 안내소 연무대 매표소,경기,수원시,2.0,3.0,4.0,60.0,0,e000541,3,여,40,1,4,3,5,2,3,6,5,6,8,1,2
312,e_e000541,화성행궁,경기,수원시,2.0,4.0,4.0,30.0,0,e000541,3,여,40,1,4,3,5,2,3,6,5,6,8,1,2
313,e_e000541,수원 화성 북서적대,경기,수원시,7.0,3.0,3.0,30.0,0,e000541,3,여,40,1,4,3,5,2,3,6,5,6,8,1,2
0,e_e000004,화성 관광열차 안내소 연무대 매표소,경기,수원시,2.0,3.0,4.0,60.0,0,e000004,3,남,40,7,5,3,5,4,5,4,2,5,1,1,2
1,e_e000004,창룡문,경기,수원시,2.0,4.0,4.0,30.0,0,e000004,3,남,40,7,5,3,5,4,5,4,2,5,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22355,g_g013131,평화광장,전남,목포시,1.0,5.0,5.0,60.0,1,g013131,4,남,20,1,6,3,2,3,3,6,6,6,5,1,1
22356,g_g013131,목포 갓바위,전남,목포시,1.0,4.0,4.0,60.0,1,g013131,4,남,20,1,6,3,2,3,3,6,6,6,5,1,1
22359,h_h003280,법주사,충북,보은군,2.0,5.0,5.0,90.0,0,h003280,3,여,40,5,1,4,4,4,4,4,4,4,4,1,0
22360,h_h003280,정이품송,충북,보은군,1.0,5.0,5.0,30.0,0,h003280,3,여,40,5,1,4,4,4,4,4,4,4,4,1,0


time: 16 ms (started: 2024-08-03 02:14:37 +09:00)


In [42]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostRegressor

def calculate_recall_at_k(actual, predicted, k):
    actual_set = set(actual)
    predicted_set = set(predicted[:k])
    intersection_count = len(actual_set & predicted_set)
    recall_at_k = intersection_count / len(actual_set) if actual_set else 0
    return recall_at_k

def preprocess_data(X, cat_features):
    X = X.copy()
    for cat_feature in cat_features:
        X[cat_feature] = X[cat_feature].astype(str)
    num_features = X.columns.difference(cat_features + ['TRAVEL_ID', 'TRAVELER_ID', 'DGSTFN'])
    X[num_features] = X[num_features].apply(pd.to_numeric, errors='coerce')
    X = X.drop(columns=['TRAVEL_ID', 'TRAVELER_ID'], errors='ignore')
    if 'DGSTFN' in X.columns:
        X = X.drop(columns=['DGSTFN'], errors='ignore')
    return X, num_features

def train_and_evaluate_model(X_train, y_train, X_val, y_val, cat_features, params):
    model = CatBoostRegressor(
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        subsample=params['subsample'],
        colsample_bylevel=params['colsample_bylevel'],
        cat_features=cat_features,
        random_state=params['random_state'],
        silent=True  # Suppress output
    )
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    recall_list = [
        calculate_recall_at_k([X_val.iloc[i]['VISIT_AREA_NM']], X_val['VISIT_AREA_NM'][np.argsort(y_pred)[::-1]], 10)
        for i in range(len(X_val))
    ]
    return np.mean(recall_list) if recall_list else 0

def cross_validation(X_train, y_train, cv, iterations, random_state):
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=random_state)
    cat_features = ['VISIT_AREA_NM', 'SIDO', 'GUNGU', 'VISIT_AREA_TYPE_CD', 'TRAVEL_MISSION_PRIORITY', 'AGE_GRP', 'GENDER']

    folds = [
        (X_train.iloc[train_index].reset_index(drop=True), y_train.iloc[train_index].reset_index(drop=True), 
         X_train.iloc[test_index].reset_index(drop=True), y_train.iloc[test_index].reset_index(drop=True))
        for train_index, test_index in tqdm(skf.split(X_train, y_train), total=cv)
    ]
    
    np.random.seed(random_state)
    initial = 0

    for iter in tqdm(range(iterations)):
        params = {
            'n_estimators': np.random.choice(np.arange(700, 1301, 50)),
            'max_depth': np.random.choice(np.arange(5, 11, 1)),
            'subsample': np.random.choice(np.arange(0.75, 1.0001, 0.05)),
            'colsample_bylevel': np.random.choice(np.arange(0.8, 1.0001, 0.05)),
            'random_state': random_state
        }
        
        final_recall = []
        for j in range(cv):
            combine_indices = list(range(cv))
            combine_indices.remove(j)
            
            X_new_train = pd.concat([folds[i][0] for i in combine_indices], axis=0).reset_index(drop=True)
            y_new_train = np.concatenate([folds[i][1] for i in combine_indices])

            X_new_train, num_features = preprocess_data(X_new_train, cat_features)
            X_val, _ = preprocess_data(folds[j][2], cat_features)

            recall_at_10 = train_and_evaluate_model(X_new_train, y_new_train, X_val, folds[j][3], cat_features, params)
            final_recall.append(recall_at_10)
        
        recallat10 = np.mean(final_recall) if final_recall else 0
        
        if recallat10 > initial:
            initial = recallat10
            final_params = params
    
    print('최종 parameter은 :', final_params)
    return initial

# Example usage
# initial = cross_validation(X_train, y_train, cv=5, iterations=10, random_state=42)


time: 0 ns (started: 2024-08-03 02:14:37 +09:00)


여행지 별 방문자 수나 재방문 건수, 방문자 수 별 구간 화 변환
방법, 재방문 건수 별 구간 화 변환 방법보다 방문자 수 및 재방문 건수
가 모두 고려된 Matrix 방법이 가장 우수한 결과를 나타냈다.

https://oasis.dcollection.net/public_resource/pdf/200000001484_20240803022912.pdf

In [43]:
initial = cross_validation(X_train, y_train, cv=5, iterations=10, random_state=42)

100%|██████████| 10/10 [1:26:53<00:00, 521.40s/it]

최종 parameter은 : {'n_estimators': 1050, 'max_depth': 9, 'subsample': 0.9500000000000002, 'colsample_bylevel': 0.8500000000000001, 'random_state': 42}
time: 1h 26min 54s (started: 2024-08-03 02:14:37 +09:00)


In [44]:
initial

0.008797947603480367

time: 0 ns (started: 2024-08-03 03:43:56 +09:00)


In [45]:
joblib.dump({'n_estimators': 1050, 'max_depth': 9, 'subsample': 0.9500000000000002, 'colsample_bylevel': 0.8500000000000001, 'random_state': 42}, os.path.join(path, 'catboost_model_params.pkl'))

['./catboost_model_params.pkl']

time: 16 ms (started: 2024-08-03 03:48:09 +09:00)


In [50]:
catboost_params = joblib.load(os.path.join(path, 'catboost_model_params.pkl'))
modelc = CatBoostRegressor(**catboost_params)

{'loss_function': 'RMSE',
 'subsample': 0.9500000000000002,
 'max_depth': 9,
 'n_estimators': 1050,
 'colsample_bylevel': 0.8500000000000001,
 'random_state': 42}

time: 0 ns (started: 2024-08-03 03:49:02 +09:00)
